<a href="https://colab.research.google.com/github/yan2727/quant-macro/blob/main/quant_macro_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import pandas as pd
import numpy as np


# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
data = pwt1001.loc[pwt1001['country'].isin(["France","Germany","Canada","Italy","Japan","United Kingdom","United States"])][['year', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna']]
data = data.loc[(data['year'] >= 1995) & (data['year'] <= 2019)].dropna()

# Calculate additional variables
data['y_pc'] = np.log(data['rgdpna'] / data['emp'])  # GDP per worker log　(y=Y/N)
data['k_pc'] = np.log(data['rkna'] / data['emp'])  # Capital per worker (k=K/N)

data['alpha'] = 1 - data['labsh'] # Capital share 資本支配率
data['y_n'] = data['rgdpna'] / data['emp']  # Y/N
data['hours'] = data['emp'] * data['avh']  # L
data['tfp_term'] = data['rtfpna'] ** (1 / (1 - data['alpha']))  # A^(1/(1-alpha))
data['cap_term'] = (data['rkna'] / data['rgdpna']) ** (data['alpha'] / (1 - data['alpha']))  # (K/Y)^(alpha/(1-alpha))
data['lab_term'] = data['hours'] / data['pop']  # L/N


# Order by year
data = data.sort_values('year')

# Group by isocode
grouped_data = data.groupby('countrycode')

# Calculate growth rates and Solow residual
data['g'] = (grouped_data['y_pc'].diff() * 100)  # Growth rate of GDP per capita
data['cd']= data['alpha'] * (grouped_data['k_pc'].diff() * 100) #Capital Deepning
data['tg']= data['g'] - data['cd'] #TFP Growth

# Remove missing values
data = data.dropna()

# Calculate summary statistics
summary = data.groupby('countrycode').agg({'g': 'mean',
                                       'tg': "mean",
                                          'cd': 'mean', 
                                          'alpha': 'mean'})

# Calculate additional summary statistics
summary['Growth Rate'] = summary['g']
summary['Capital Deepening'] = summary['cd']
summary['TFP Growth'] = summary['tg']
summary['TFP Share'] = summary['TFP Growth'] / summary['Growth Rate']
summary['Capital Share'] = summary['alpha']


# Print output
print(summary)

                    g        tg        cd     alpha  Growth Rate  \
countrycode                                                        
CAN          0.887700  0.253687  0.634013  0.346190     0.887700   
DEU          0.661055  0.223337  0.437718  0.370730     0.661055   
FRA          0.829236  0.260418  0.568817  0.381194     0.829236   
GBR          1.015544  0.442543  0.573001  0.412993     1.015544   
ITA         -0.037181 -0.571016  0.533835  0.484835    -0.037181   
JPN          0.710956  0.079903  0.631053  0.430601     0.710956   
USA          1.536985  0.756660  0.780325  0.393260     1.536985   

             Capital Deepening  TFP Growth  TFP Share  Capital Share  
countrycode                                                           
CAN                   0.634013    0.253687   0.285780       0.346190  
DEU                   0.437718    0.223337   0.337849       0.370730  
FRA                   0.568817    0.260418   0.314046       0.381194  
GBR                   0.573001  